# Explore Parliament Speeches

1. remove duplicates
2. handle missings
3.. procedural or otherwise nonimportant speeches? - DONE

In [23]:
import pandas as pd

### Import complete dataset of speeches

In [24]:
data = pd.read_csv("../data/complete_data.csv")

In [25]:
data.head()

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction
0,f7f58b13-85f3-424a-6864-08da102a68d8,11001235,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11001235,Wolfgang,Kubicki,FDP,FDP
1,d0c84378-49ab-47ce-61aa-08da102a68d8,11001938,Das ist der Fall. – Ich sehe keine weiteren Vo...,19,1,3,11001938,Dr. Wolfgang,Schäuble,CDU,NaN
2,9aff56c3-41cd-4e6b-633b-08da0f22a008,11002190,"Guten Morgen, liebe Kolleginnen und Kollegen! ...",19,1,1,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN
3,05ebe489-0cff-4575-661e-08da0f22a008,11002190,Die unterbrochene Sitzung ist wieder eröffnet....,19,1,6,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN
4,901e2196-b575-4fea-5fd2-08da102a68d8,11002190,Herr Bundespräsident! Verehrte Kolleginnen und...,19,1,4,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN


In [26]:
import string
# defining normalization of text: converting to lowercase and removing punctuation
def normalize_text(input_str:str):
    return input_str.lower().translate(str.maketrans('','', string.punctuation))


### Create some helper columns for identification of speeches to include

In [27]:
# normalize (remove punctuation, to lower)
data['normalized'] = data['speech_text'].apply(normalize_text)
# Tokenized speech
data['tokenized'] = data['normalized'].str.split()
# length of speech
data['speech_length'] = data['tokenized'].apply(len)

### Initial inspection

In [28]:
tokenized = data['tokenized'].dropna()

# 1. Total number of speeches
num_speeches = len(tokenized)

# 2. Average speech length (in words)
avg_length = tokenized.apply(len).mean()

# 3. Maximum and minimum speech length
max_length = tokenized.apply(len).max()
min_length = tokenized.apply(len).min()

# 4. Vocabulary size (unique words across all utterances)
all_words = [word for sublist in tokenized for word in sublist]
vocab_size = len(set(all_words))

# 5. Total word count
total_words = len(all_words)

# Show results
print(f"Total number of speech: {num_speeches}")
print(f"Average speech length (words): {avg_length:.2f}")
print(f"Max speech length: {max_length} words")
print(f"Min speech length: {min_length} words")
print(f"Total word count: {total_words}")
print(f"Vocabulary size: {vocab_size}")


Total number of speech: 32589
Average speech length (words): 580.42
Max speech length: 9349 words
Min speech length: 1 words
Total word count: 18915184
Vocabulary size: 284582


### Duplicates?

In [29]:
print(len(data['speech_id'].unique())) # unique speech identifier
print(len(data))

32589
32589


-> no duplicates

### Missings?

In [30]:
na_counts = data.isna().sum()
na_counts = na_counts[na_counts > 0]
print(na_counts)

party         30
fraction    3434
dtype: int64


-> keep only rows with party info (30 rows without party info are no relevant loss to the dataset), fraction information won't be needed

In [31]:
data = data[data['party'].notna()].reset_index(drop=True)

### Importing meta information to speeches

In [65]:
df_agenda = pd.read_csv("../data/agenda_item_info.csv")

### Collection of Speeches to be excluded

#### identify via keywords in agenda item title

In [66]:
contains_matches = df_agenda['AgendaItemTitle'].str.contains(
    "Wahl|Eröffnung|Eidesleistung|Bekanntgabe|Fragestunde|Befragung|Nationalhymne|Geschäftsordnung", na=False # formal / procedural content
)

# filter mask
mask = contains_matches

# store matches in new df
to_drop = df_agenda.loc[mask, [
    'AgendaItemTitle', 
    'LegislaturePeriod', 
    'Number', 
    'AgendaItemNumber', 
    'Order', 
    'DateOnly'
]].rename(columns={
    'LegislaturePeriod': 'Legislature',
    'Number': 'Protocol_Nr'
}).reset_index(drop=True)


In [67]:
to_drop

,AgendaItemTitle,Legislature,Protocol_Nr,AgendaItemNumber,Order,DateOnly
0,Eröffnung der Sitzung durch den Alterspräsidenten,19,1,1,1,2017-10-24
1,Beschlussfassung über die Geschäftsordnung,19,1,2,2,2017-10-24
2,Wahl des Präsidenten,19,1,3,3,2017-10-24
3,Wahl der Stellvertreter des Präsidenten,19,1,6,6,2017-10-24
4,Nationalhymne,19,1,7,7,2017-10-24
...,...,...,...,...,...,...
288,Fragestunde,20,87,2,2,2023-03-01
289,Wahl der Mitglieder des Vermittlungsausschusses,20,9,2,1,2021-12-16
290,Befragung der Bundesregierung (BMI und BMZ),20,90,1,1,2023-03-15
291,Fragestunde,20,90,2,2,2023-03-15


#### match titles via lagislature, protocol and agendaitemnumber/order to speech content

In [68]:
# MultiIndex-Mask
mask = data.set_index(['legislative_period', 'protocol_nr', 'agenda_item_number']).index.isin(
    to_drop.set_index(['Legislature', 'Protocol_Nr', 'Order']).index
)

matched_data = data[mask].reset_index(drop=True)


In [69]:
matched_data

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction,normalized,tokenized,speech_length
0,f7f58b13-85f3-424a-6864-08da102a68d8,11001235,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11001235,Wolfgang,Kubicki,FDP,FDP,herr präsident ich nehme die wahl an\ndann beg...,"[herr, präsident, ich, nehme, die, wahl, an, d...",22
1,d0c84378-49ab-47ce-61aa-08da102a68d8,11001938,Das ist der Fall. – Ich sehe keine weiteren Vo...,19,1,3,11001938,Dr. Wolfgang,Schäuble,CDU,NaN,das ist der fall – ich sehe keine weiteren vor...,"[das, ist, der, fall, –, ich, sehe, keine, wei...",572
2,9aff56c3-41cd-4e6b-633b-08da0f22a008,11002190,"Guten Morgen, liebe Kolleginnen und Kollegen! ...",19,1,1,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN,guten morgen liebe kolleginnen und kollegen ne...,"[guten, morgen, liebe, kolleginnen, und, kolle...",273
3,05ebe489-0cff-4575-661e-08da0f22a008,11002190,Die unterbrochene Sitzung ist wieder eröffnet....,19,1,6,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN,die unterbrochene sitzung ist wieder eröffnet\...,"[die, unterbrochene, sitzung, ist, wieder, erö...",257
4,917c3517-5bc1-4763-5c38-08da0f22a008,11003124,"Herr Präsident, ich nehme die Wahl an.\nDann b...",19,1,6,11003124,Hans-Peter,Friedrich,CSU,CDU/CSU,herr präsident ich nehme die wahl an\ndann beg...,"[herr, präsident, ich, nehme, die, wahl, an, d...",14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,a4088e45-d1d5-4124-a617-3e778850db38,11005095,"Frau Präsidentin, vielen Dank. – Sehr geehrte ...",20,90,1,11005095,Lamya,Kaddor,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen,frau präsidentin vielen dank – sehr geehrte fr...,"[frau, präsidentin, vielen, dank, –, sehr, gee...",146
4194,7ae715a6-990e-421f-be78-618f6ea4d32c,11005260,"Frau Ministerin, für den Bund führen Sie als B...",20,90,1,11005260,Janine,Wissler,DIE LINKE.,Die Linke,frau ministerin für den bund führen sie als bu...,"[frau, ministerin, für, den, bund, führen, sie...",166
4195,6931ac71-d28c-45f4-9ac0-589e88d38546,11005266,"Gut, haushalterisch muss man die Prioritäten d...",20,90,1,11005266,Nicolas,Zippelius,CDU,CDU/CSU,gut haushalterisch muss man die prioritäten da...,"[gut, haushalterisch, muss, man, die, prioritä...",61
4196,2df162e1-b2c6-46eb-bc99-4ce63a694a7c,11005289,Sie haben die Reformvorschläge auf EU-Ebene an...,20,90,1,11005289,Clara,Bünger,DIE LINKE.,Die Linke,sie haben die reformvorschläge auf euebene ang...,"[sie, haben, die, reformvorschläge, auf, euebe...",40


In [70]:
# how many rows affected?
data.value_counts(['legislative_period', 'protocol_nr', 'agenda_item_number']).reset_index(name='count').sort_values('count', ascending=False)


,legislative_period,protocol_nr,agenda_item_number,count
0,19,70,1,70
1,20,59,1,64
2,19,175,1,62
3,19,94,1,62
4,19,136,1,62
...,...,...,...,...
2859,19,236,22,1
2863,19,215,7,1
2880,19,1,1,1
2881,19,166,20,1


In [71]:
mask = ~data.set_index(['legislative_period', 'protocol_nr', 'agenda_item_number']).index.isin(
    to_drop.set_index(['Legislature', 'Protocol_Nr', 'Order']).index
)

# Gefiltertes DataFrame
data_cleaned = data[mask].reset_index(drop=True)

In [72]:
data_cleaned

,speech_id,speaker_id,speech_text,legislative_period,protocol_nr,agenda_item_number,speakerId,firstName,lastName,party,fraction,normalized,tokenized,speech_length
0,901e2196-b575-4fea-5fd2-08da102a68d8,11002190,Herr Bundespräsident! Verehrte Kolleginnen und...,19,1,4,11002190,Alterspräsident Dr. Hermann,Otto Solms,FDP,NaN,herr bundespräsident verehrte kolleginnen und ...,"[herr, bundespräsident, verehrte, kolleginnen,...",2206
1,a914bfea-8684-4446-0640-08da0f05b642,11002738,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,11002738,Hans,Michelbach,CSU,CDU/CSU,herr präsident kolleginnen und kollegen die ba...,"[herr, präsident, kolleginnen, und, kollegen, ...",520
2,02a4928b-8f4e-45d8-5f3e-08da102a68d8,11003422,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,11003422,Ingrid,Arndt-Brauer,SPD,SPD,sehr geehrter herr präsident liebe kolleginnen...,"[sehr, geehrter, herr, präsident, liebe, kolle...",693
3,ef59b72f-b33d-43da-5de1-08da0f22a008,11003646,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,11003646,Antje,Tillmann,CDU,CDU/CSU,herr präsident liebe kolleginnen und kollegen ...,"[herr, präsident, liebe, kolleginnen, und, kol...",710
4,8dd9b261-a22c-431b-279d-08da0d9e2391,11003837,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,11003837,Gerhard,Schick,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen,herr präsident liebe kolleginnen und kollegen ...,"[herr, präsident, liebe, kolleginnen, und, kol...",752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28356,49664a2c-adde-4492-bfd5-97d1a99a83f7,11005228,Frau Präsidentin! Meine Damen! Meine Herren! I...,20,92,1,11005228,Till,Steffen,BÜNDNIS 90/DIE GRÜNEN,Bündnis 90 / Die Grünen,frau präsidentin meine damen meine herren ich ...,"[frau, präsidentin, meine, damen, meine, herre...",495
28357,851ade38-70d2-46c1-8125-8d799c7602db,11005251,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,20,92,2,11005251,Carmen,Wegge,SPD,SPD,sehr geehrte frau präsidentin sehr geehrte kol...,"[sehr, geehrte, frau, präsidentin, sehr, geehr...",673
28358,63673dff-7423-4a3d-a613-05c0ee1b23c1,11005251,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,20,92,5,11005251,Carmen,Wegge,SPD,SPD,sehr geehrte frau präsidentin sehr geehrte kol...,"[sehr, geehrte, frau, präsidentin, sehr, geehr...",573
28359,ef2ca5c9-7242-42b6-b574-b22e40cf0253,11005263,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,92,3,11005263,Mareike Lotte,Wulf,CDU,CDU/CSU,sehr geehrter herr präsident liebe kolleginnen...,"[sehr, geehrter, herr, präsident, liebe, kolle...",596


In [73]:
data_cleaned = data_cleaned[data_cleaned['speech_length'] > 200]
data_cleaned = data_cleaned[data_cleaned['speech_length'] < 1501]

### remove speeches by unobserved parties

In [74]:
data_cleaned['party'].value_counts()

party
SPD                      5785
CDU                      5203
AfD                      3728
BÜNDNIS 90/DIE GRÜNEN    3550
FDP                      3460
DIE LINKE.               2798
CSU                      2136
Plos                      262
LKR                        82
Die PARTEI                 30
SSW                        20
Name: count, dtype: int64

In [75]:
parties_to_remove = ['Plos', 'LKR', 'Die PARTEI', 'SSW']

data_cleaned = data_cleaned[~data_cleaned['party'].isin(parties_to_remove)].reset_index(drop=True)

In [76]:
data_cleaned['party'].value_counts()

party
SPD                      5785
CDU                      5203
AfD                      3728
BÜNDNIS 90/DIE GRÜNEN    3550
FDP                      3460
DIE LINKE.               2798
CSU                      2136
Name: count, dtype: int64

In [77]:
tokenized = data_cleaned['tokenized'].dropna()

# 1. Total number of speeches
num_speeches = len(tokenized)

# 2. Average speech length (in words)
avg_length = tokenized.apply(len).mean()
med_length = tokenized.apply(len).median()
mod_length = tokenized.apply(len).mode()


# 3. Maximum and minimum speech length
max_length = tokenized.apply(len).max()
min_length = tokenized.apply(len).min()

# 4. Vocabulary size (unique words across all utterances)
all_words = [word for sublist in tokenized for word in sublist]
vocab_size = len(set(all_words))

# 5. Total word count
total_words = len(all_words)

# Show results
print(f"Total number of speech: {num_speeches}")
print(f"Average speech length (mean): {avg_length:.2f}")
print(f"Average speech length (median): {med_length:.2f}")
print(f"Average speech length (mode): {mod_length.iloc[0]:.2f}")
print(f"Max speech length: {max_length} words")
print(f"Min speech length: {min_length} words")
print(f"Total word count: {total_words}")
print(f"Vocabulary size: {vocab_size}")


Total number of speech: 26660
Average speech length (mean): 646.09
Average speech length (median): 616.00
Average speech length (mode): 560.00
Max speech length: 1500 words
Min speech length: 201 words
Total word count: 17224652
Vocabulary size: 272858


### formatting

In [85]:
df_agenda.columns

Index(['Unnamed: 0', 'agenda_item_title', 'AgendaItemNumber',
       'agenda_item_number', 'AgendaItemDate', 'ProtocolId', 'date',
       'ProtocolDate', 'legislative_period', 'protocol_nr', 'SessionTitle',
       'AgendaItemsCount', 'MongoId', 'Id'],
      dtype='object')

In [86]:
data_cleaned.columns

Index(['speech_id', 'speaker_id', 'speech_text', 'legislative_period',
       'protocol_nr', 'agenda_item_number', 'speakerId', 'firstName',
       'lastName', 'party', 'fraction', 'normalized', 'tokenized',
       'speech_length'],
      dtype='object')

In [87]:
df_agenda = df_agenda.rename(columns={
    'LegislaturePeriod': 'legislative_period',
    'Number': 'protocol_nr',
    'Order':'agenda_item_number',
    'DateOnly':'date',
    'AgendaItemTitle':'agenda_item_title'
})

In [110]:
data_merged = data_cleaned.merge(
    df_agenda[['legislative_period', 'protocol_nr', 'agenda_item_number', 'agenda_item_title','date']],
    on=['legislative_period', 'protocol_nr', 'agenda_item_number'],
    how='left'
)


#### combine CDU/CSU, clean party names

In [111]:
data_merged['party'].value_counts()

party
SPD                      5785
CDU                      5203
AfD                      3728
BÜNDNIS 90/DIE GRÜNEN    3550
FDP                      3460
DIE LINKE.               2798
CSU                      2136
Name: count, dtype: int64

In [112]:
data_merged['party'] = data_merged['party'].replace(['CDU', 'CSU'], 'CDU/CSU')
data_merged['party'] = data_merged['party'].replace(['BÜNDNIS 90/DIE GRÜNEN'], 'GRÜNE')
data_merged['party'] = data_merged['party'].replace(['DIE LINKE.'], 'LINKE')

In [113]:
data_merged['party'].value_counts()

party
CDU/CSU    7339
SPD        5785
AfD        3728
GRÜNE      3550
FDP        3460
LINKE      2798
Name: count, dtype: int64

#### combine Names

In [114]:
data_merged['full_name'] = data_merged['firstName'] + ' ' + data_merged['lastName']


#### drop helper columns

In [115]:
data_merged = data_merged.drop(columns=['speech_id','speaker_id','speakerId','fraction','normalized','tokenized','speech_length','firstName','lastName'])

In [116]:
data_merged

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,CDU/CSU,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,CDU/CSU,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar
...,...,...,...,...,...,...,...,...
26655,Frau Präsidentin! Meine Damen! Meine Herren! I...,20,92,1,GRÜNE,Änderung des Bundeswahlgesetzes,2023-03-17,Till Steffen
26656,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,20,92,2,SPD,"Schutz vor sexuellem Missbrauch, IP-Adr.-speic...",2023-03-17,Carmen Wegge
26657,Sehr geehrte Frau Präsidentin! Sehr geehrte Ko...,20,92,5,SPD,Schutz hinweisgebender Personen,2023-03-17,Carmen Wegge
26658,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,92,3,CDU/CSU,Vereinbarte Debatte zum Internationalen Frauentag,2023-03-17,Mareike Lotte Wulf


In [117]:
data_merged.to_csv("../data/final_data.csv")